In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing as sk_preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from tqdm import tqdm_notebook
from xverse.transformer import MonotonicBinning
from category_encoders.woe import WOEEncoder


RM_PROJECT_LIB_PATH = "../risk-management-project2"
if RM_PROJECT_LIB_PATH not in sys.path:
    sys.path.append(RM_PROJECT_LIB_PATH)
import project_lib
from project_lib.default_stats import simulate_total_loss

In [2]:
TRAIN_DATA = pd.read_csv('../data/train.csv')
TEST_DATA = pd.read_csv("../data/val.csv")

/Users/nialeksandrov/virtualenvs/risks/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7,36) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
TRAIN_DATA['ts'] = pd.to_datetime(TRAIN_DATA.issue_d)
TEST_DATA['ts'] = pd.to_datetime(TEST_DATA.issue_d)

TRAIN_DATA = TRAIN_DATA.sort_values('ts').reset_index().drop('index', axis=1)
TEST_DATA = TEST_DATA.sort_values('ts').reset_index().drop('index', axis=1)

TRAIN_DATA.drop('ts', axis=1, inplace=True)
TEST_DATA.drop('ts', axis=1, inplace=True)

In [4]:
# TRAIN_DATA.issue_d = pd.to_datetime(TRAIN_DATA.issue_d)
# TEST_DATA.issue_d = pd.to_datetime(TEST_DATA.issue_d)
# TRAIN_DATA = TRAIN_DATA.sort_values('issue_d').reset_index().drop('index', axis=1)
# TEST_DATA = TEST_DATA.sort_values('issue_d').reset_index().drop('index', axis=1)

# Задача 1

Задача - построить интерпретируемый скоринг. Для этого:

In [5]:
TASK1_PIKLES_FOLDER = "../pickles/task_1"

In [6]:
TRAIN_FULL_WOE = TRAIN_DATA.copy()
TEST_WOE = TEST_DATA.copy()

TRAIN_FULL_WOE = TRAIN_FULL_WOE.drop(["issue_d", "earliest_cr_line"], axis=1)
TEST_WOE = TEST_WOE.drop(["issue_d", "earliest_cr_line"], axis=1)

TRAIN_FULL_WOE = project_lib.preprocessing.prepare_data(TRAIN_FULL_WOE)

Remove all_util because it have more than 20% (72.0) of NaNs
Remove annual_inc_joint because it have more than 20% (99.0) of NaNs
Remove desc because it have more than 20% (89.0) of NaNs
Remove dti_joint because it have more than 20% (99.0) of NaNs
Remove il_util because it have more than 20% (76.0) of NaNs
Remove inq_fi because it have more than 20% (72.0) of NaNs
Remove inq_last_12m because it have more than 20% (72.0) of NaNs
Remove member_id because it have more than 20% (100.0) of NaNs
Remove verification_status_joint because it have more than 20% (99.0) of NaNs
Drop columns ['id', 'policy_code', 'zip_code']
Fill annual_inc NaNs with rolling mean
Fill avg_cur_bal NaNs with rolling mean
Fill dti NaNs with rolling mean
Fill inq_last_6mths NaNs with rolling mean
Fill pub_rec NaNs with rolling mean
Fill pub_rec_bankruptcies NaNs with rolling mean
Fill tax_liens NaNs with rolling mean
Fill emp_length NaNs with 'unk'
Fill emp_title NaNs with 'unk'
Fill title NaNs with 'unk'


In [7]:
TEST_WOE = project_lib.preprocessing.prepare_data(TEST_WOE)

Remove annual_inc_joint because it have more than 20% (91.0) of NaNs
Remove desc because it have more than 20% (100.0) of NaNs
Remove dti_joint because it have more than 20% (91.0) of NaNs
Remove member_id because it have more than 20% (100.0) of NaNs
Remove verification_status_joint because it have more than 20% (91.0) of NaNs
Drop columns ['id', 'policy_code', 'zip_code']
Fill all_util NaNs with rolling mean
Fill avg_cur_bal NaNs with rolling mean
Fill dti NaNs with rolling mean
Fill il_util NaNs with rolling mean
Fill emp_length NaNs with 'unk'
Fill emp_title NaNs with 'unk'


In [8]:
# for col_name in TEST_WOE.columns:
#     TEST_WOE.loc[TEST_WOE.loc[:, col_name].isna(), col_name] = TEST_WOE.loc[:, col_name].mode()

In [9]:
TRAIN_FULL_WOE.dropna(inplace=True)
TEST_WOE.dropna(inplace=True)

## 1.a

С помощью библиотеки xverse преобразуем непрерывные переменные на интервалы, преобразовав их в ординальные с помощью функции MonotonicBinning

In [10]:
target_col_name = "default"
real_features_names = list(TRAIN_FULL_WOE.select_dtypes(np.number).columns)
cat_features_names = list(TRAIN_FULL_WOE.select_dtypes("object").columns)
cat_features_names.remove("emp_title")
cat_features_names.remove("title")
all_features_names = real_features_names + cat_features_names

In [11]:
res = train_test_split(TRAIN_FULL_WOE.loc[:, all_features_names], TRAIN_FULL_WOE.loc[:, target_col_name], 
                       test_size=0.25, shuffle=False)
TRAIN_TRAIN_WOE, TRAIN_VAL_WOE, TRAIN_TRAIN_WOE_TARGET, TRAIN_VAL_WOE_TARGET = res
TEST_WOE, TEST_WOE_TARGET = TEST_WOE.loc[:, all_features_names], TEST_WOE.loc[:, target_col_name]
TRAIN_FULL_WOE, TRAIN_FULL_WOE_TARGET = TRAIN_FULL_WOE.loc[:, all_features_names],\
                                        TRAIN_FULL_WOE.loc[:, target_col_name]

In [12]:
real_features_binner = MonotonicBinning(force_bins=10, feature_names=list(real_features_names))
TRAIN_TRAIN_WOE = real_features_binner.fit_transform(TRAIN_TRAIN_WOE, TRAIN_TRAIN_WOE_TARGET)
TRAIN_VAL_WOE = real_features_binner.transform(TRAIN_VAL_WOE)
TEST_WOE = real_features_binner.transform(TEST_WOE)
TRAIN_FULL_WOE =real_features_binner.fit_transform(TRAIN_FULL_WOE, TRAIN_FULL_WOE_TARGET)

Заполним пропуски самыми популряными катеоргиями

In [13]:
most_popular_values = TRAIN_TRAIN_WOE.mode(axis=0)
for feature_name in TRAIN_TRAIN_WOE.columns:
    most_popular_value = most_popular_values[feature_name][0]
    
    is_na_in_test = TRAIN_VAL_WOE.loc[:, feature_name].isna()
    if is_na_in_test.any():
        TRAIN_VAL_WOE.loc[is_na_in_test, feature_name] = most_popular_value
        
    is_na_in_final_test = TEST_WOE.loc[:, feature_name].isna()
    if is_na_in_final_test.any():
        TEST_WOE.loc[is_na_in_final_test, feature_name] = most_popular_value

In [14]:
for feature_name in tqdm_notebook(all_features_names):
    label_encoder = LabelEncoder()
    TRAIN_TRAIN_WOE.loc[:, feature_name] = label_encoder.fit_transform(TRAIN_TRAIN_WOE.loc[:, feature_name])
    TRAIN_VAL_WOE.loc[:, feature_name] = label_encoder.transform(TRAIN_VAL_WOE.loc[:, feature_name])
    TEST_WOE.loc[:, feature_name] = label_encoder.transform(TEST_WOE.loc[:, feature_name])
    
    label_encoder = LabelEncoder()
    TRAIN_FULL_WOE.loc[:, feature_name] = label_encoder.fit_transform(TRAIN_FULL_WOE.loc[:, feature_name])

In [15]:
train_data_for_iv = TRAIN_FULL_WOE.copy()
woe_encoder = project_lib.utils.VanillaWoe()
woe_encoder.fit(train_data_for_iv, TRAIN_FULL_WOE_TARGET)
train_data_for_iv = woe_encoder.transform(train_data_for_iv)

../risk-management-project2/project_lib/utils.py:83: RuntimeWarning: divide by zero encountered in double_scalars
  return np.log(fraction_of_non_events / fraction_of_events)


In [16]:
woe_encoder = WOEEncoder(cols=TRAIN_TRAIN_WOE.columns)
woe_encoder.fit(TRAIN_TRAIN_WOE, TRAIN_TRAIN_WOE_TARGET)
TRAIN_VAL_WOE = woe_encoder.transform(TRAIN_VAL_WOE)

woe_encoder.fit(TRAIN_TRAIN_WOE, TRAIN_TRAIN_WOE_TARGET)
TEST_WOE = woe_encoder.transform(TEST_WOE)

TRAIN_TRAIN_WOE = project_lib.task1_utils.encode_train_data_with_woe(TRAIN_TRAIN_WOE, TRAIN_TRAIN_WOE_TARGET)
TRAIN_FULL_WOE = project_lib.task1_utils.encode_train_data_with_woe(TRAIN_FULL_WOE, TRAIN_FULL_WOE_TARGET)

In [17]:
train_data_for_iv["target"] = TRAIN_FULL_WOE_TARGET
def calc_iv(x, total_events_amount, total_non_events_amount):
    events_amount = x.sum()
    non_events_amount = x.shape[0] - events_amount
    fraction_of_events = events_amount / total_events_amount
    fraction_of_non_events = non_events_amount / total_non_events_amount
    iv = (fraction_of_non_events - fraction_of_events)
    iv = iv * x.name
    if np.isinf(iv):
        return 0
    else:
        return iv

information_values = pd.DataFrame(index=train_data_for_iv.columns[:-1], columns=["Information Value"])
total_events_amount = train_data_for_iv["target"].sum()
total_non_events_amount = train_data_for_iv.shape[0] - total_events_amount
for col_name in tqdm_notebook(information_values.index):
    iv = train_data_for_iv.groupby([col_name])["target"]\
                          .apply(calc_iv, total_events_amount, total_non_events_amount).sum()
    information_values.loc[col_name, "Information Value"] = iv

train_data_for_iv.drop(["target"], axis=1, inplace=True)
information_values.sort_values("Information Value", ascending=False)

,Information Value
int_rate,0.452599
term,0.207414
fico_range_high,0.115846
fico_range_low,0.115846
dti,0.0753507
verification_status,0.0531599
avg_cur_bal,0.0478452
funded_amnt,0.0325512
annual_inc,0.0295156
home_ownership,0.0281266


Вычислим коэффициент Джини для каждого фактора

In [18]:
features_gini_scores = pd.DataFrame(index=TRAIN_TRAIN_WOE.columns, columns=["Gini Train", "Gini Test"])
for col_name in tqdm_notebook(TRAIN_TRAIN_WOE.columns):
    file_path = os.path.join(TASK1_PIKLES_FOLDER, f"{col_name}_small_lr.pkl")
    X_train = TRAIN_TRAIN_WOE.loc[:, col_name].to_numpy(copy=True).reshape(-1, 1)
    model = project_lib.utils.fit_or_upload_logistic_regression(file_path, X_train, TRAIN_TRAIN_WOE_TARGET)
    
    y_train_predicted = model.predict_proba(TRAIN_TRAIN_WOE.loc[:, col_name].to_numpy(copy=True)\
                                                           .reshape(-1, 1))[:, 1]
    train_gini = project_lib.metrics.gini_score(TRAIN_TRAIN_WOE_TARGET, y_train_predicted)
    y_test_predicted = model.predict_proba(TRAIN_VAL_WOE.loc[:, col_name].to_numpy(copy=True)\
                                                        .reshape(-1, 1))[:, 1]
    test_gini = project_lib.metrics.gini_score(TRAIN_VAL_WOE_TARGET, y_test_predicted)
    
    features_gini_scores.loc[col_name, "Gini Train"] = train_gini
    features_gini_scores.loc[col_name, "Gini Test"] = test_gini
features_gini_scores.sort_values(["Gini Test"], ascending=False)

,Gini Train,Gini Test
int_rate,0.343266,0.351791
fico_range_high,0.175109,0.18784
fico_range_low,0.175109,0.18784
dti,0.159798,0.147043
term,0.242361,0.145828
avg_cur_bal,0.101506,0.131197
verification_status,0.0900337,0.130544
home_ownership,0.0781958,0.108563
annual_inc,0.0951251,0.101197
funded_amnt,0.0969975,0.0871483


# 1. b/c

In [19]:
initial_features = features_gini_scores["Gini Test"].sort_values(ascending=False).index

In [20]:
backward_features = project_lib.model_selection.backward_lr(TRAIN_TRAIN_WOE, TRAIN_TRAIN_WOE_TARGET, 
                                                            TRAIN_VAL_WOE, TRAIN_VAL_WOE_TARGET, 
                                                            TASK1_PIKLES_FOLDER, initial_features, "small_lr")

removing application_type feature improved score
cur score - 0.40336538964994206



In [21]:
forward_features = project_lib.model_selection.forward_lr(TRAIN_TRAIN_WOE, TRAIN_TRAIN_WOE_TARGET, 
                                                          TRAIN_VAL_WOE, TRAIN_VAL_WOE_TARGET, 
                                                          TASK1_PIKLES_FOLDER, initial_features, "small_lr")

including int_rate feature improved score. Cur set of features: ['int_rate']
cur score - 0.35179071483728164
including fico_range_high feature improved score. Cur set of features: ['int_rate', 'fico_range_high']
cur score - 0.36282254040130524
including dti feature improved score. Cur set of features: ['int_rate', 'fico_range_high', 'fico_range_low', 'dti']
cur score - 0.36441206905415013
including term feature improved score. Cur set of features: ['int_rate', 'fico_range_high', 'fico_range_low', 'dti', 'term']
cur score - 0.37129770511402915
including avg_cur_bal feature improved score. Cur set of features: ['int_rate', 'fico_range_high', 'fico_range_low', 'dti', 'term', 'avg_cur_bal']
cur score - 0.38406639880559434
including verification_status feature improved score. Cur set of features: ['int_rate', 'fico_range_high', 'fico_range_low', 'dti', 'term', 'avg_cur_bal', 'verification_status']
cur score - 0.38618167944698323
including home_ownership feature improved score. Cur set of fe

In [22]:
print(f'# backward features - {len(backward_features)}')
print(f'# forward features - {len(forward_features)}')
LR_FEATURES = forward_features.copy()

# backward features - 20
# forward features - 20


На основании Information Value, значений индивидуального коэффициента Gini оставим фичи ['int_rate', 'fico_range_high', 'fico_range_low', 'dti', 'term', 'avg_cur_bal', 'verification_status', 'home_ownership', 'annual_inc', 'funded_amnt', 'installment'], потому что после installment - слишком маленькие значения IV и Gini, а также в forward selection - нет значимого прироста в качестве. 

In [23]:
LR_FEATURES = ['int_rate', 'fico_range_high', 'fico_range_low', 'dti', 'term', 'avg_cur_bal',
               'verification_status', 'home_ownership', 'annual_inc', 'funded_amnt', 'installment']

Оба подхода отобрали одинаковое кол-во признаков (все). Означает, что каждый признак важен в некоторой степени

## Оценим лучшую модель на отложенной и тестовой выборке

In [24]:
file_path = project_lib.utils.create_model_file_path(TRAIN_TRAIN_WOE.columns, LR_FEATURES, TASK1_PIKLES_FOLDER,
                                                     "small_lr")
TRAIN_TRAIN_LR_MODEL = project_lib.utils.fit_or_upload_logistic_regression(file_path, 
                                                                           TRAIN_TRAIN_WOE.loc[:, LR_FEATURES], 
                                                                           TRAIN_TRAIN_WOE_TARGET)
predicts_val_logreg = TRAIN_TRAIN_LR_MODEL.predict_proba(TEST_WOE.loc[:, LR_FEATURES])
gini_score = project_lib.metrics.gini_score(TEST_WOE_TARGET, predicts_val_logreg[:, 1])
print(f' Полученный Gini: {gini_score : .4f}')

 Полученный Gini:  0.3697


In [25]:
file_path = project_lib.utils.create_model_file_path(TRAIN_FULL_WOE.columns, LR_FEATURES, TASK1_PIKLES_FOLDER,
                                                     "full_lr")
TRAIN_FULL_LR_MODEL = project_lib.utils.fit_or_upload_logistic_regression(file_path, 
                                                                          TRAIN_FULL_WOE.loc[:, LR_FEATURES], 
                                                                          TRAIN_FULL_WOE_TARGET)
predicts_val_logreg = TRAIN_FULL_LR_MODEL.predict_proba(TEST_WOE.loc[:, LR_FEATURES])
gini_score = project_lib.metrics.gini_score(TEST_WOE_TARGET, predicts_val_logreg[:, 1])
print(f' Полученный Gini: {gini_score : .4f}')

 Полученный Gini:  0.3723


Outputs

In [26]:
temp = (TRAIN_TRAIN_WOE, TRAIN_VAL_WOE, LR_FEATURES, TRAIN_TRAIN_LR_MODEL, TRAIN_FULL_LR_MODEL, TRAIN_FULL_WOE, 
        TEST_WOE, TRAIN_TRAIN_WOE_TARGET, TRAIN_VAL_WOE_TARGET, TEST_WOE_TARGET)

# Задание 2

В качестве альтернативной модели будем использовать градиентный бустинг над решающими деревьями в имплементации бибилотеки Catboost с последующей калибровкой прогнозов.

In [27]:
import catboost

In [28]:
train_pd = TRAIN_DATA.copy(deep=True)

## Анализ данных

In [29]:
for col in train_pd.columns:
    print(f'{col} : {len(train_pd[col].unique())}')

addr_state : 51
all_util : 166
annual_inc : 57517
annual_inc_joint : 1640
application_type : 2
avg_cur_bal : 72303
default : 2
desc : 124408
disbursement_method : 2
dti : 5276
dti_joint : 2444
earliest_cr_line : 716
emp_length : 12
emp_title : 345417
fico_range_high : 48
fico_range_low : 48
funded_amnt : 1511
home_ownership : 6
id : 1128702
il_util : 221
initial_list_status : 2
inq_fi : 30
inq_last_12m : 39
inq_last_6mths : 29
installment : 77068
int_rate : 595
issue_d : 115
member_id : 1
policy_code : 1
pub_rec : 39
pub_rec_bankruptcies : 13
purpose : 14
tax_liens : 37
term : 2
title : 63155
verification_status : 3
verification_status_joint : 2
zip_code : 942


In [30]:
cat_features = ['zip_code', 'verification_status_joint', 'verification_status', 'term', 'tax_liens', 'purpose',
                'initial_list_status', 'home_ownership', 'disbursement_method', 'addr_state', 'application_type',
                'emp_length', 'issue_d_month', 'earliest_cr_line_month']
text_features = ['title', 'emp_title', 'desc']

In [31]:
def prepare_cols_with_dates(df):
    df.earliest_cr_line.fillna('nan-nan', inplace=True)
    for col in ['issue_d', 'earliest_cr_line']:
        month_year =  df[col].apply(lambda x: x.split('-'))
        month_year = pd.DataFrame(month_year.tolist(), columns=[f'{col}_month', f'{col}_year'])
        df = pd.concat([df, month_year], axis=1)
        df.drop([col], axis=1, inplace=True)
    return df

def fillna_cats(df, cat_features):
    for cat_feature in cat_features:
        df[cat_feature] = df[cat_feature].fillna('NaN')
        df[cat_feature] = df[cat_feature].astype('str')
    df.drop(text_features, axis=1, inplace=True)
    return df

In [32]:
TRAIN_FULL_CATBOOST = train_pd.copy(deep=True)

In [33]:
TRAIN_FULL_CATBOOST = prepare_cols_with_dates(TRAIN_FULL_CATBOOST)
TRAIN_FULL_CATBOOST = fillna_cats(TRAIN_FULL_CATBOOST, cat_features)

## Разделим датасет для обучения модели и ее калибровки 

In [34]:
class catboost_calibration():
    def __init__(self, catboost_model, logreg_model):
        self.catboost_model = catboost_model
        self.logreg_model = logreg_model
        
    def predict_proba(self, X_test):
        predict_calibration = self.catboost_model.predict_proba(X_test)
        return self.logreg_model.predict_proba(
            predict_calibration[:, 1].reshape(-1, 1))

In [35]:
TRAIN_TRAIN_CATBOOST = TRAIN_FULL_CATBOOST[:TRAIN_TRAIN_WOE.index.max()]
TRAIN_VAL_CATBOOST = TRAIN_FULL_CATBOOST[TRAIN_TRAIN_WOE.index.max():]

In [36]:
def fit_catboost_with_calibration(train_dataset : pd.DataFrame,
                                  cat_features: list) -> tuple([catboost.CatBoostClassifier,
                                                                        LogisticRegression]):
    train_dataset = train_dataset.copy(deep=True)
    X_train, X_test_for_calibration, y_train, y_test_for_calibration = train_test_split(
    train_dataset.drop('default', axis=1),
    train_dataset.default, 
    test_size=0.1,
    random_state=42,
    shuffle=False
    )
    
    model = catboost.CatBoostClassifier(cat_features=cat_features,
                                    class_weights=[1.0, X_train.shape[0]/y_train.sum()],
                                    random_state=42,
                                    depth=4,
                                    l2_leaf_reg=20,
                                    learning_rate=0.05,
                                    random_strength = 5,
                                    bagging_temperature=5)

    model.fit(X_train, #примерное вреия выполнения -- 10 минут
              y=y_train,
              verbose=False,
              cat_features=cat_features)
    predict_calibration = model.predict_proba(X_test_for_calibration)
    LR = LogisticRegression(random_state=42, solver='lbfgs')
    LR.fit(predict_calibration[:, 1].reshape(-1, 1), y_test_for_calibration)
 
    return model, LR

In [37]:
try:
    with open('../pickles/catboost/TRAIN_TRAIN_CATBOOST_MODEL.model', 'rb') as f:
        TRAIN_TRAIN_CATBOOST_MODEL = pickle.load(f)
except:
    train_catboost, train_lr = fit_catboost_with_calibration(TRAIN_TRAIN_CATBOOST, cat_features)
    pickle.dump(train_catboost, open('../pickles/catboost/train_catboost.model', 'wb'))
    pickle.dump(train_lr, open('../pickles/catboost/train_lr.model', 'wb'))
    
    TRAIN_TRAIN_CATBOOST_MODEL = catboost_calibration(train_catboost, train_lr)
    pickle.dump(TRAIN_TRAIN_CATBOOST_MODEL,
            open('../pickles/catboost/TRAIN_TRAIN_CATBOOST_MODEL.model', 'wb'))

/Users/nialeksandrov/virtualenvs/risks/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/nialeksandrov/virtualenvs/risks/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [38]:
try:
    with open('../pickles/catboost/TRAIN_FULL_CATBOOST_MODEL.model', 'rb') as f:
        TRAIN_FULL_CATBOOST_MODEL = pickle.load(f)
    with open('../pickles/catboost/full_catboost.model', 'rb') as f:
        full_catboost = pickle.load(f)
except:
    full_catboost, full_lr = fit_catboost_with_calibration(TRAIN_FULL_CATBOOST, cat_features)
    pickle.dump(full_catboost, open('../pickles/catboost/full_catboost.model', 'wb'))
    pickle.dump(full_lr, open('../pickles/catboost/full_lr.model', 'wb'))
    
    TRAIN_FULL_CATBOOST_MODEL = catboost_calibration(full_catboost, full_lr)    
    pickle.dump(TRAIN_FULL_CATBOOST_MODEL,
            open('../pickles/catboost/TRAIN_FULL_CATBOOST_MODEL.model', 'wb'))

## Оценим модель на валидационной выборке

In [39]:
TEST_CATBOOST = TEST_DATA.copy(deep=True)

In [40]:
TEST_CATBOOST = prepare_cols_with_dates(TEST_CATBOOST)
TEST_CATBOOST = fillna_cats(TEST_CATBOOST, cat_features)

In [41]:
X_test_val = TEST_CATBOOST.drop('default', axis=1)
y_test_val = TEST_CATBOOST.default

In [42]:
predict_cal = TRAIN_FULL_CATBOOST_MODEL.predict_proba(X_test_val)

In [43]:
predict_val = full_catboost.predict_proba(X_test_val)

In [44]:
print(f' Полученный Gini: {project_lib.metrics.gini_score(y_test_val, predict_cal[:, 1]) : .4f}')

 Полученный Gini:  0.4212


In [45]:
from sklearn.calibration import calibration_curve
from matplotlib import pyplot as plt

In [46]:
fraction_of_positives, mean_predicted_value = \
    calibration_curve(y_test_val, predict_val[:, 1], n_bins=8)

plt.figure(figsize = (15, 10))
plt.plot(mean_predicted_value, fraction_of_positives, "s-", label='catboost')

fraction_of_positives, mean_predicted_value = \
    calibration_curve(y_test_val, predict_cal[:, 1], n_bins=8)
plt.plot(mean_predicted_value, fraction_of_positives, "s-", label='calibration')

plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")


plt.ylabel("Fraction of positives")
plt.ylim([-0.05, 1.05])
plt.legend(loc="lower right")
plt.title('Calibration plots  (reliability curve)')

Text(0.5, 1.0, 'Calibration plots  (reliability curve)')

# Задание 3

На основе построенных моделей постройте модель оценки внутренних рейтингов (выделите не меньше 7 не-дефолтных рейтинговых категорий).

a. Проще всего это сделать разбиением предсказания модели на интервалы.

b. Оцените вероятности дефолта для каждого внутреннего рейтинга вместе с доверительными интервалами. Также оцените общую дефолтность по всей выборке
вместе с доверительным интервалом.

c. Стоит ли объединить какие-нибудь рейтинговые категории из построенных в одну?
Почему?

Возьмем наши предикты на валидации:

В качестве не-дефолтных категорий будем рассматривать категории с вероятностью дефолта выше 50%.

Даже после калибровки моделей confidence в прогнозах будет отличаться, поэтому более правильно будет использовать две линейки категорий (под каждую из моделей).

In [47]:
import string

In [48]:
def calculate_category(score: float, cutted_bins) -> str:
    idx = (cutted_bins > score).argmax()
    if idx == 0:
        return string.ascii_uppercase[0]
    else:
        return string.ascii_uppercase[len(cutted_bins) - idx - 1]
    return string.ascii_uppercase[idx]

In [49]:
def calculate_new_category(score: float, cutted_bins, labels) -> str:
    labels = labels[::-1]
    cutted_bins = cutted_bins[::-1]
    idx = (cutted_bins > score).argmax()
    if score > cutted_bins[-1]:
        return labels[-1]
    return labels[idx]

## Посчитаем для первой модели

In [50]:
y_test_val = TEST_WOE_TARGET

In [51]:
df_for_category_logreg = pd.DataFrame(
            {'predict_proba': predicts_val_logreg[:, 0], 'default': y_test_val}
)

In [52]:
df_for_category_logreg = df_for_category_logreg[df_for_category_logreg.predict_proba > 0.5] #вероятность не-дефолта = 1 - вероятность дефолта

In [53]:
bins = 15
df_for_category_logreg["bins"], cutted_bins = pd.cut(
    df_for_category_logreg["predict_proba"], 
    bins = bins, retbins=True)
df_for_category_logreg['category'] = df_for_category_logreg.bins.cat.rename_categories(
      reversed(list(string.ascii_uppercase)[:bins]))

In [54]:
df_for_category_logreg.head()

,predict_proba,default,bins,category
1,0.825344,False,"(0.82, 0.852]",E
2,0.823097,False,"(0.82, 0.852]",E
3,0.833522,False,"(0.82, 0.852]",E
4,0.668381,False,"(0.66, 0.692]",J
5,0.722260,True,"(0.692, 0.724]",I


In [55]:
plt.figure(figsize = (20, 10))
plt.scatter(
    df_for_category_logreg.groupby('category').default.mean().index,
    df_for_category_logreg.groupby('category').default.mean().values)
plt.title('Процент дефолтов по категориям', fontsize=18)

Text(0.5, 1.0, 'Процент дефолтов по категориям')

In [56]:
plt.figure(figsize = (20, 10))
plt.scatter(
    df_for_category_logreg.groupby('category').default.sum().index,
    df_for_category_logreg.groupby('category').default.sum().values)
plt.title('Количество дефолтов по категориям', fontsize=18)

Text(0.5, 1.0, 'Количество дефолтов по категориям')

In [57]:
print(f'Доля дефолтов во всех не-дефолтных категориях: {df_for_category_logreg.default.mean() : .4}')

Доля дефолтов во всех не-дефолтных категориях:  0.2585


Мы видим, что во всех категориях не нулевое количество дефолтов. Для получения вероятности дефолта для каждого внутреннего рейтинга вместе с доверительными интервалами воспользуемся бустрэпом.

In [58]:
TEST_WOE.shape

(240863, 21)

In [59]:
TRAIN_TRAIN_WOE.shape

(797178, 21)

In [60]:
plt.figure(figsize = (20, 10))
plt.grid()

N_ITER = 100

data_agg: np.ndarray = np.zeros(shape=(N_ITER, len(cutted_bins) - 1))
data_overall: np.ndarray = np.zeros(N_ITER)    
    
for i in range(0, N_ITER):
    sample_index = np.random.choice(range(0, len(y_test_val)), len(y_test_val))
    y_samples = y_test_val.iloc[sample_index]   
    X_samples = TEST_WOE.iloc[sample_index][LR_FEATURES]
    predict_cal_samples = TRAIN_FULL_LR_MODEL.predict_proba(X_samples)
    df_for_category_ = pd.DataFrame(
            {'predict_proba': predict_cal_samples[:, 0], 'default': y_samples}
    )
    df_for_category_ = df_for_category_[df_for_category_.predict_proba > 0.5]
    df_for_category_['category'] = df_for_category_.predict_proba.apply(lambda x: 
                                                                        calculate_category(x, cutted_bins))
    #print(f'Доля дефолтов во всех не-дефолтных категориях: {df_for_category_.default.mean() : .4}')
    plt.scatter(
    df_for_category_.groupby('category').default.mean().index,
    df_for_category_.groupby('category').default.mean().values)
    plt.title('Процент дефолтов в разрезе по категориям', fontsize=18)
    
    data_agg[i, :] = df_for_category_.groupby('category').default.mean().values
    data_overall[i] = df_for_category_.default.mean()

In [61]:
fig = plt.figure(1, figsize=(20, 15))
plt.grid()

ax = fig.add_subplot(111)
plt.title('Процент дефолтов по не-дефолтным категориям в целом', fontsize=18)

_ = ax.boxplot(data_overall, whis=[2.5, 97.5])

/Users/nialeksandrov/virtualenvs/risks/lib/python3.7/site-packages/ipykernel_launcher.py:4: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  after removing the cwd from sys.path.


In [62]:
fig = plt.figure(1, figsize=(20, 15))
plt.grid()

ax = fig.add_subplot(111)
plt.title('Процент дефолтов в разрезе по категориям', fontsize=18)

labels = ["{}".format(int(i)) for i in ax.get_xticks()]
ax.set_xticklabels(df_for_category_.groupby('category').default.mean().index.values,
                  fontsize=14)
_ = ax.boxplot(data_agg, whis=[2.5, 97.5])

/Users/nialeksandrov/virtualenvs/risks/lib/python3.7/site-packages/ipykernel_launcher.py:4: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  after removing the cwd from sys.path.


In [63]:
print(f''' Процент дефолтов по всем не-дефолтным категориям находится в доверительном 99% интервале 
      [ {np.percentile(data_overall, 0.005) : .6} , {np.percentile(data_overall, 0.095) : .6} ]
      ''')

 Процент дефолтов по всем не-дефолтным категориям находится в доверительном 99% интервале 
      [  0.256514 ,  0.256514 ]
      


In [64]:
cats = df_for_category_.groupby('category').default.mean().index.values

results_df = pd.DataFrame({'category' : cats, 
              '2.5%': [np.percentile(data_agg[:, i], 0.025) for i in range(data_agg.shape[1])], 
              '97.5%':[np.percentile(data_agg[:, i], 0.975) for i in range(data_agg.shape[1])],
              'low_boundary': list(reversed(cutted_bins[1:]))})

In [65]:
print(f'Процент дефолтов в разрезе по категориям (доверительный 95% интервал):')
results_df

Процент дефолтов в разрезе по категориям (доверительный 95% интервал):


,category,2.5%,97.5%,low_boundary
0,A,0.052749,0.053005,0.980286
1,B,0.103696,0.103980,0.948268
2,C,0.154336,0.154833,0.916250
3,D,0.200627,0.201723,0.884232
4,E,0.242997,0.243415,0.852214
5,F,0.282695,0.283274,0.820196
6,G,0.317996,0.318049,0.788178
7,H,0.348020,0.350150,0.756160
8,I,0.379881,0.380383,0.724142
9,J,0.397412,0.398239,0.692125


Объединим пересекающиеся категории.

In [66]:
intersection_with_prev = np.zeros(results_df.shape[0])
for i in range(1, results_df.shape[0]):
    intersection_with_prev[i] = results_df.iloc[i, 1] < results_df.iloc[i-1, 2]

In [67]:
results_df['intersect_with_prev'] = intersection_with_prev

In [68]:
results_df

,category,2.5%,97.5%,low_boundary,intersect_with_prev
0,A,0.052749,0.053005,0.980286,0.0
1,B,0.103696,0.103980,0.948268,0.0
2,C,0.154336,0.154833,0.916250,0.0
3,D,0.200627,0.201723,0.884232,0.0
4,E,0.242997,0.243415,0.852214,0.0
5,F,0.282695,0.283274,0.820196,0.0
6,G,0.317996,0.318049,0.788178,0.0
7,H,0.348020,0.350150,0.756160,0.0
8,I,0.379881,0.380383,0.724142,0.0
9,J,0.397412,0.398239,0.692125,0.0


In [69]:
idx_to_remove = results_df[results_df.intersect_with_prev == 1].index

In [70]:
for idx in idx_to_remove:
    results_df.iloc[idx-1, 2] = results_df.iloc[idx, 2]
    results_df.iloc[idx-1, 0] += '_' + results_df.iloc[idx, 0]
    results_df.iloc[idx-1, 3] = results_df.iloc[idx, 3]
results_df = results_df.drop(results_df.index[idx_to_remove])

In [71]:
results_df

,category,2.5%,97.5%,low_boundary,intersect_with_prev
0,A,0.052749,0.053005,0.980286,0.0
1,B,0.103696,0.103980,0.948268,0.0
2,C,0.154336,0.154833,0.916250,0.0
3,D,0.200627,0.201723,0.884232,0.0
4,E,0.242997,0.243415,0.852214,0.0
5,F,0.282695,0.283274,0.820196,0.0
6,G,0.317996,0.318049,0.788178,0.0
7,H,0.348020,0.350150,0.756160,0.0
8,I,0.379881,0.380383,0.724142,0.0
9,J,0.397412,0.398239,0.692125,0.0


In [72]:
print(f'Итоговые категории: ')
print(f'{results_df.iloc[-1, 0]} : [ {0.5 :.4f} : {results_df.iloc[-1, 3]:.4f} ] ')
for i in reversed(range(1, results_df.shape[0] - 1)):
      print(f'{results_df.iloc[i, 0]} : [ {results_df.iloc[i+1, 3] :.4f} : {results_df.iloc[i, 3] : .4f} ] ')
print(f'{results_df.iloc[0, 0]} : [ {results_df.iloc[1, 3] : .4f} : {1. : .4f} ] ')

Итоговые категории: 
O : [ 0.5000 : 0.5320 ] 
N : [ 0.5320 :  0.5641 ] 
M : [ 0.5641 :  0.5961 ] 
L : [ 0.5961 :  0.6281 ] 
K : [ 0.6281 :  0.6601 ] 
J : [ 0.6601 :  0.6921 ] 
I : [ 0.6921 :  0.7241 ] 
H : [ 0.7241 :  0.7562 ] 
G : [ 0.7562 :  0.7882 ] 
F : [ 0.7882 :  0.8202 ] 
E : [ 0.8202 :  0.8522 ] 
D : [ 0.8522 :  0.8842 ] 
C : [ 0.8842 :  0.9162 ] 
B : [ 0.9162 :  0.9483 ] 
A : [  0.9483 :  1.0000 ] 


In [73]:
logreg_labels = results_df.category.values
logreg_cutted_bins = results_df.low_boundary.values

In [74]:
def calculate_new_category_logreg(score: float,
                                  cutted_bins=logreg_cutted_bins, 
                                  labels=logreg_labels) -> str:
    '''
    score -- вероятность не-дефолта
    '''
    labels = labels[::-1]
    cutted_bins = cutted_bins[::-1]
    idx = (cutted_bins > score).argmax()
    if score > cutted_bins[-1]:
        return labels[-1]
    return labels[idx]

In [75]:
y_test_val = TEST_DATA.default

In [76]:
df_for_category = pd.DataFrame(
            {'predict_proba': predict_cal[:, 0], 'default': y_test_val}
)

In [77]:
df_for_category = df_for_category[df_for_category.predict_proba > 0.5] #вероятность не-дефолта = 1 - вероятность дефолта

In [78]:
bins = 15
df_for_category["bins"], cutted_bins = pd.cut(df_for_category["predict_proba"], bins = bins, retbins=True)
df_for_category['category'] = df_for_category.bins.cat.rename_categories(
      reversed(list(string.ascii_uppercase)[:bins]))

In [79]:
df_for_category.head()

,predict_proba,default,bins,category
0,0.803040,False,"(0.787, 0.819]",F
1,0.791214,False,"(0.787, 0.819]",F
2,0.738253,False,"(0.723, 0.755]",H
3,0.745090,False,"(0.723, 0.755]",H
4,0.584313,False,"(0.564, 0.596]",M


In [80]:
plt.figure(figsize = (20, 10))
plt.scatter(
    df_for_category.groupby('category').default.mean().index,
    df_for_category.groupby('category').default.mean().values)
plt.title('Процент дефолтов по категориям', fontsize=18)

Text(0.5, 1.0, 'Процент дефолтов по категориям')

In [81]:
plt.figure(figsize = (20, 10))
plt.scatter(
    df_for_category.groupby('category').default.sum().index,
    df_for_category.groupby('category').default.sum().values)
plt.title('Количество дефолтов по категориям', fontsize=18)

Text(0.5, 1.0, 'Количество дефолтов по категориям')

In [82]:
print(f'Доля дефолтов во всех не-дефолтных категориях: {df_for_category.default.mean() : .4}')

Доля дефолтов во всех не-дефолтных категориях:  0.2466


Мы видим, что во всех категориях не нулевое количество дефолтов. Для получения вероятности дефолта для каждого внутреннего рейтинга вместе с доверительными интервалами воспользуемся бустрэпом.

In [83]:
plt.figure(figsize = (20, 10))
plt.grid()

N_ITER = 100

data_agg: np.ndarray = np.zeros(shape=(N_ITER, len(cutted_bins) - 1))
data_overall: np.ndarray = np.zeros(N_ITER)    
    
for i in range(0, N_ITER):
    sample_index = np.random.choice(range(0, len(y_test_val)), len(y_test_val))
    y_samples = y_test_val.iloc[sample_index]   
    X_samples = X_test_val.iloc[sample_index, :]
    predict_cal_samples = TRAIN_FULL_CATBOOST_MODEL.predict_proba(X_samples)
    df_for_category_ = pd.DataFrame(
            {'predict_proba': predict_cal_samples[:, 0], 'default': y_samples}
    )
    df_for_category_ = df_for_category_[df_for_category_.predict_proba > 0.5]
    df_for_category_['category'] = df_for_category_.predict_proba.apply(lambda x: 
                                                                        calculate_category(x, cutted_bins))
    plt.scatter(
    df_for_category_.groupby('category').default.mean().index,
    df_for_category_.groupby('category').default.mean().values)
    plt.title('Процент дефолтов в разрезе по категориям', fontsize=18)
    
    data_agg[i, :] = df_for_category_.groupby('category').default.mean().values
    data_overall[i] = df_for_category_.default.mean()

In [84]:
fig = plt.figure(1, figsize=(20, 15))
plt.grid()

ax = fig.add_subplot(111)
plt.title('Процент дефолтов по не-дефолтным категориям в целом', fontsize=18)

_ = ax.boxplot(data_overall, whis=[2.5, 97.5])

/Users/nialeksandrov/virtualenvs/risks/lib/python3.7/site-packages/ipykernel_launcher.py:4: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  after removing the cwd from sys.path.


In [85]:
fig = plt.figure(1, figsize=(20, 15))
plt.grid()

ax = fig.add_subplot(111)
plt.title('Процент дефолтов в разрезе по категориям', fontsize=18)

labels = ["{}".format(int(i)) for i in ax.get_xticks()]
ax.set_xticklabels(df_for_category_.groupby('category').default.mean().index.values,
                  fontsize=14)
_ = ax.boxplot(data_agg, whis=[2.5, 97.5])

/Users/nialeksandrov/virtualenvs/risks/lib/python3.7/site-packages/ipykernel_launcher.py:4: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  after removing the cwd from sys.path.


In [86]:
print(f''' Процент дефолтов по всем не-дефолтным категориям находится в доверительном 99% интервале 
      [ {np.percentile(data_overall, 0.005) : .6} , {np.percentile(data_overall, 0.095) : .6} ]
      ''')

 Процент дефолтов по всем не-дефолтным категориям находится в доверительном 99% интервале 
      [  0.244062 ,  0.244136 ]
      


In [87]:
cats = df_for_category_.groupby('category').default.mean().index.values

results_df = pd.DataFrame({'category' : cats, 
              '2.5%': [np.percentile(data_agg[:, i], 0.025) for i in range(data_agg.shape[1])], 
              '97.5%':[np.percentile(data_agg[:, i], 0.975) for i in range(data_agg.shape[1])],
              'low_boundary': reversed(cutted_bins[1:])})

In [88]:
print(f'Процент дефолтов в разрезе по категориям (доверительный 95% интервал):')
results_df

Процент дефолтов в разрезе по категориям (доверительный 95% интервал):


,category,2.5%,97.5%,low_boundary
0,A,0.032157,0.035366,0.978315
1,B,0.076403,0.076961,0.946428
2,C,0.110897,0.111053,0.914540
3,D,0.143959,0.144793,0.882652
4,E,0.166737,0.167197,0.850765
5,F,0.195518,0.195528,0.818877
6,G,0.223201,0.224094,0.786990
7,H,0.247142,0.247457,0.755102
8,I,0.273405,0.274575,0.723214
9,J,0.313773,0.314785,0.691327


Объединим пересекающиеся категории.

In [89]:
intersection_with_prev = np.zeros(results_df.shape[0])
for i in range(1, results_df.shape[0]):
    intersection_with_prev[i] = results_df.iloc[i, 1] < results_df.iloc[i-1, 2]

In [90]:
results_df['intersect_with_prev'] = intersection_with_prev

In [91]:
results_df

,category,2.5%,97.5%,low_boundary,intersect_with_prev
0,A,0.032157,0.035366,0.978315,0.0
1,B,0.076403,0.076961,0.946428,0.0
2,C,0.110897,0.111053,0.914540,0.0
3,D,0.143959,0.144793,0.882652,0.0
4,E,0.166737,0.167197,0.850765,0.0
5,F,0.195518,0.195528,0.818877,0.0
6,G,0.223201,0.224094,0.786990,0.0
7,H,0.247142,0.247457,0.755102,0.0
8,I,0.273405,0.274575,0.723214,0.0
9,J,0.313773,0.314785,0.691327,0.0


In [92]:
idx_to_remove = results_df[results_df.intersect_with_prev == 1].index

In [93]:
for idx in idx_to_remove:
    results_df.iloc[idx-1, 2] = results_df.iloc[idx, 2]
    results_df.iloc[idx-1, 0] += '_' + results_df.iloc[idx, 0]
    results_df.iloc[idx-1, 3] = results_df.iloc[idx, 3]
results_df = results_df.drop(results_df.index[idx_to_remove])

In [94]:
results_df

,category,2.5%,97.5%,low_boundary,intersect_with_prev
0,A,0.032157,0.035366,0.978315,0.0
1,B,0.076403,0.076961,0.946428,0.0
2,C,0.110897,0.111053,0.914540,0.0
3,D,0.143959,0.144793,0.882652,0.0
4,E,0.166737,0.167197,0.850765,0.0
5,F,0.195518,0.195528,0.818877,0.0
6,G,0.223201,0.224094,0.786990,0.0
7,H,0.247142,0.247457,0.755102,0.0
8,I,0.273405,0.274575,0.723214,0.0
9,J,0.313773,0.314785,0.691327,0.0


In [95]:
print(f'Итоговые категории: ')
print(f'{results_df.iloc[-1, 0]} : [ {0.5 :.4f} : {results_df.iloc[-1, 3]:.4f} ] ')
for i in reversed(range(1, results_df.shape[0] - 1)):
      print(f'{results_df.iloc[i, 0]} : [ {results_df.iloc[i+1, 3] :.4f} : {results_df.iloc[i, 3] : .4f} ] ')
print(f'{results_df.iloc[0, 0]} : [ {results_df.iloc[1, 3] : .4f} : {1. : .4f} ] ')

Итоговые категории: 
O : [ 0.5000 : 0.5319 ] 
N : [ 0.5319 :  0.5638 ] 
M : [ 0.5638 :  0.5957 ] 
L : [ 0.5957 :  0.6276 ] 
K : [ 0.6276 :  0.6594 ] 
J : [ 0.6594 :  0.6913 ] 
I : [ 0.6913 :  0.7232 ] 
H : [ 0.7232 :  0.7551 ] 
G : [ 0.7551 :  0.7870 ] 
F : [ 0.7870 :  0.8189 ] 
E : [ 0.8189 :  0.8508 ] 
D : [ 0.8508 :  0.8827 ] 
C : [ 0.8827 :  0.9145 ] 
B : [ 0.9145 :  0.9464 ] 
A : [  0.9464 :  1.0000 ] 


In [96]:
catboost_labels = results_df.category.values
catboost_cutted_bins = results_df.low_boundary.values

In [97]:
def calculate_new_category_catboost(score: float,
                                  cutted_bins=catboost_cutted_bins, 
                                  labels=catboost_labels) -> str:
    '''
    score -- вероятность не-дефолта
    '''
    labels = labels[::-1]
    cutted_bins = cutted_bins[::-1]
    idx = (cutted_bins > score).argmax()
    if score > cutted_bins[-1]:
        return labels[-1]
    return labels[idx]

Примеры вычисления категорий. 

In [98]:
calculate_new_category_catboost(0.82)

'E'

In [99]:
calculate_new_category_logreg(0.82)

'F'

# 4.Проведите валидацию обеих построенных моделей, используя валидационную выборку.

4.a - в одних осях постройте гистограммы распределений скоринговых баллов для
дефолтёров и не-дефолтёров.

In [100]:
final_lr = LogisticRegression(solver='lbfgs')
final_lr.fit(X_train, y_train)

NameError: name 'y_train' is not defined

In [ ]:
from project_lib.fourth_task_utils import plot_scoring_histograms

In [ ]:
predicts_val_logreg = final_lr.predict_proba(X_final_test)[:, 1]

In [ ]:
default_scores = np.round((1 - predicts_val_logreg)* 1000)

In [ ]:
plot_scoring_histograms(default_scores[y_final_test == 0], default_scores[y_final_test == 1])

4.b - Постройте CAP-кривую, отметив также идеальную модель и случайную, вычислите AUC
и коэффициент Джини. Не забудьте доверительные интервалы для всех значений. Что
можно сказать о качестве модели?

In [ ]:
from project_lib.fourth_task_utils import plot_cap_curve

LogReg:

In [ ]:
arg = pd.DataFrame({
    'default': y_final_test,
    'probability_default': predicts_val_logreg
})

In [ ]:
plot_cap_curve(arg)

In [ ]:
from project_lib.fourth_task_utils import bootstrap_ci

In [ ]:
for ci_level in [95, 99]:
    gini_low_level, gini_up_level = bootstrap_ci(final_lr, X_final_test, y_final_test,
                                                 ci_level, 100, 0.7,
                                                 lambda true, pred: 2*roc_auc_score(true, pred) - 1)
    auc_low_level, auc_up_level = bootstrap_ci(final_lr, X_final_test, y_final_test,
                                               ci_level, 100, 0.7,
                                               lambda true, pred: roc_auc_score(true, pred))
    print(f'gini {ci_level} conf int -> [{gini_low_level:.2f}; {gini_up_level:.2f}]')
    print(f'auc {ci_level} conf int -> [{auc_low_level:.2f}; {auc_up_level:.2f}]')
    print()

Catboost:

4.c - Для каждого фактора, вошедшего в модель, оцените его индивидуальную значимость,
посчитав коэффициент Джини. Не забудьте доверительные интервалы

In [ ]:
from project_lib.fourth_task_utils import calc_gini_importances_for_features, plot_feature_importances

LogReg:

In [ ]:
log_reg_importances = calc_gini_importances_for_features(final_lr, X_train, y_train, X_final_test, y_final_test)

In [ ]:
log_reg_importances.sort_values('gini solo feature', ascending=False)

In [ ]:
plot_feature_importances(log_reg_importances)

Catboost:

4.d Определить какой инкрементальный вклад вносит каждый из факторов в итоговый коэффициент Джини модели. 

LogReg:

In [ ]:
from project_lib.fourth_task_utils import calc_feature_contribution

In [ ]:
res_table = calc_feature_contribution(final_lr, X_train, y_train, X_final_test, y_final_test)

In [ ]:
plt.figure(figsize=(15, 8))
plt.title('feature gini contribution')
res_table = res_table.sort_values('gini contribution', ascending=False)
plt.bar(res_table.index, res_table['gini contribution'] * 100)
plt.xticks(rotation='90')
plt.ylabel('gini contribution (%)')
plt.grid()
plt.show()

Catboost:

4.e Оцените динамику коэффициента Джини во времени. Убедитесь в том, что модель стабильно ранжирует наблюдения на всех временных срезах, и отсутствуют периоды, в которых ранжирующая способность модели недостаточна 

LogReg:

In [ ]:
from project_lib.fourth_task_utils import plot_time_stability

In [ ]:
final_lr.fit(X_train, y_train)

In [ ]:
plot_time_stability(final_lr, issue_d_final_test, X_final_test, y_final_test)

Catboost:

4.f Оцените экономический смысл разбиения всех переменных на именно такие категории. Гистограмма — это распределение всех заёмщиков по категориям (какой процент всех имеет именно такое значение), а линия — процент дефолтов в этой категории. 

In [ ]:
from project_lib.fourth_task_utils import plot_categories_val_test

In [ ]:
plot_categories_val_test(final_lr, X_test, y_test, X_final_test, y_final_test, calculate_new_category_logreg)

In [ ]:
logreg_cutted_bins

4.g Оцените, насколько реализованная дефолтность на валидационной выборке отличается от предсказанной: i. При помощи модели оцените среднюю дефолтность на валидационной выборке вместе с доверительными интервалами. ii. Постройте иллюстрацию (только цвета поприличнее, пожалуйста). 

In [ ]:
from project_lib.fourth_task_utils import plot_default_comparison

LogReg:

In [ ]:
rate_95 = bootstrap_ci(final_lr, X_final_test, y_final_test, 95, 100, 0.7, lambda _, preds: (preds >= 0.5).mean())
rate_99 = bootstrap_ci(final_lr, X_final_test, y_final_test, 99, 100, 0.7, lambda _, preds: (preds >= 0.5).mean())

In [ ]:
predicted_rate = default_rate(y_final_test, final_lr.predict_proba(X_final_test)[:, 1])
true_rate = (y_final_test.sum() / y_final_test.shape[0])
plot_default_comparison(predicted_rate, true_rate, rate_95, rate_99)

Catboost:

4.h То же самое, но в разрезе по внутренним рейтингам. 

In [ ]:
from project_lib.fourth_task_utils import plot_default_comparison_by_category

In [ ]:
plot_default_comparison_by_category(final_lr, X_final_test, y_final_test, calculate_new_category_logreg)

4.i Сравните разделяющую способность модели на тестовой и валидационной выборках. Не забудьте доверительные интервалы.

In [ ]:
from project_lib.fourth_task_utils import calc_score_with_ci, plot_val_test_gini

In [ ]:
plt.hist(final_lr.predict_proba(X_final_test)[:, 0])

LogReg:

In [ ]:
final_lr.fit(X_train, y_train)

In [ ]:
test_gini_ci = calc_score_with_ci(final_lr, X_test, y_test)
test_final_gini_ci = calc_score_with_ci(final_lr, X_final_test, y_final_test)

In [ ]:
test_gini_ci

In [ ]:
plot_val_test_gini(test_gini_ci, test_final_gini_ci)

Catboost:

4.j Сравните индивидуальную разделяющую способность отдельных факторов на тестовой и валидационной выборках. Не забудьте доверительные интервалы.

In [ ]:
from project_lib.fourth_task_utils import plot_val_test_comparison_by_feature

LogReg:

In [ ]:
test_importances = calc_gini_importances_for_features(final_lr, X_train, y_train, X_test, y_test)
final_test_importances = calc_gini_importances_for_features(final_lr, X_train, y_train, X_final_test, y_final_test)

In [ ]:
test_importances

In [ ]:
features = test_importances.index
for feature_subset in np.array_split(features, 5):
    plot_val_test_comparison_by_feature(
        test_importances.loc[feature_subset],
        final_test_importances.loc[feature_subset]
    )

Catboost:

# Задание 5

In [101]:
model1_name = "Logistic Regression"
model2_name = "Our Model"
exp_capital_row_name = "Expected Capital (%)"
var_row_name = "VaR (%)"
req_capital_row_name = "Required Capital (%)"

## Vasicek

In [102]:
model1 = TRAIN_FULL_LR_MODEL
model2 = TRAIN_FULL_CATBOOST_MODEL

result_no_corr = pd.DataFrame(columns=[model1_name, model2_name],
                              index=[exp_capital_row_name, var_row_name, req_capital_row_name])
result_corr = pd.DataFrame(columns=[model1_name, model2_name],
                           index=[exp_capital_row_name, var_row_name, req_capital_row_name])

model1_exposures = TEST_DATA.loc[TEST_WOE.index, "funded_amnt"].values
model1_exp_loss = project_lib.default_stats.calc_expected_loss(model1, TEST_WOE.loc[:, LR_FEATURES],
                                                               model1_exposures)
model2_exposures = TEST_DATA.loc[TEST_CATBOOST.index, "funded_amnt"].values
model2_exp_loss = project_lib.default_stats.calc_expected_loss(model2, TEST_CATBOOST.drop("default", axis=1), 
                                                               model2_exposures)

result_no_corr.loc[exp_capital_row_name, model1_name] = round(model1_exp_loss / model1_exposures.sum() * 100, 2)
result_no_corr.loc[exp_capital_row_name, model2_name] = round(model2_exp_loss / model2_exposures.sum() * 100, 2)
result_corr.loc[exp_capital_row_name, model1_name] = round(model1_exp_loss / model1_exposures.sum() * 100, 2)
result_corr.loc[exp_capital_row_name, model2_name] = round(model2_exp_loss / model2_exposures.sum() * 100, 2)

var, req_capital = project_lib.default_stats.calc_var_and_req_capital(model1, TEST_WOE.loc[:, LR_FEATURES], 0, 
                                                                      model1_exp_loss, model1_exposures, 
                                                                      0.995)
result_no_corr.loc[var_row_name, model1_name] = var 
result_no_corr.loc[req_capital_row_name, model1_name] = req_capital 

var, req_capital = project_lib.default_stats.calc_var_and_req_capital(model1, TEST_WOE.loc[:, LR_FEATURES], 
                                                                      0.06, model2_exp_loss, 
                                                                      model1_exposures, 0.995)
result_corr.loc[var_row_name, model1_name] = var 
result_corr.loc[req_capital_row_name, model1_name] = req_capital 

var, req_capital = project_lib.default_stats.calc_var_and_req_capital(model2, 
                                                                      TEST_CATBOOST.drop("default", axis=1), 0, 
                                                                      model2_exp_loss, model2_exposures, 0.995)
result_no_corr.loc[var_row_name, model2_name] = var 
result_no_corr.loc[req_capital_row_name, model2_name] = req_capital 

var, req_capital = project_lib.default_stats.calc_var_and_req_capital(model2, 
                                                                      TEST_CATBOOST.drop("default", axis=1),
                                                                      0.06, model2_exp_loss, model2_exposures,
                                                                      0.995)
result_corr.loc[var_row_name, model2_name] = var 
result_corr.loc[req_capital_row_name, model2_name] = req_capital 

In [103]:
result_no_corr

,Logistic Regression,Our Model
Expected Capital (%),20.71,27.74
VaR (%),20.71,27.74
Required Capital (%),0,-0


In [104]:
result_corr

,Logistic Regression,Our Model
Expected Capital (%),20.71,27.74
VaR (%),40.17,48.97
Required Capital (%),12.43,21.23


## Monte Carlo

In [105]:
result_no_corr = pd.DataFrame(columns=[model1_name, model2_name],
                              index=[exp_capital_row_name, var_row_name, req_capital_row_name])
result_corr = pd.DataFrame(columns=[model1_name, model2_name],
                           index=[exp_capital_row_name, var_row_name, req_capital_row_name])

monte_carlo_iters_amount = 1000
model1_losses_distribution_no_corr = np.empty(monte_carlo_iters_amount)
model1_losses_distribution_corr = np.empty(monte_carlo_iters_amount)
model1_predicted_probs = model1.predict_proba(TEST_WOE.loc[:, LR_FEATURES])[:, 1]

model2_losses_distribution_no_corr = np.empty(monte_carlo_iters_amount)
model2_losses_distribution_corr = np.empty(monte_carlo_iters_amount)
model2_predicted_probs = model2.predict_proba(TEST_CATBOOST.drop("default", axis=1))[:, 1]

for i in tqdm_notebook(range(monte_carlo_iters_amount)):
    scenario = np.random.randn()
    
    model1_losses_distribution_no_corr[i] = simulate_total_loss(model1_predicted_probs, model1_exposures)
    model2_losses_distribution_no_corr[i] = simulate_total_loss(model2_predicted_probs, model2_exposures)
    
    model1_corr_probs = project_lib.default_stats.vasicek_cond_probs(model1_predicted_probs, scenario, 0.06)
    model1_losses_distribution_corr[i] = simulate_total_loss(model1_corr_probs, model1_exposures)
    model2_corr_probs = project_lib.default_stats.vasicek_cond_probs(model2_predicted_probs, scenario, 0.06)
    model2_losses_distribution_corr[i] = simulate_total_loss(model2_corr_probs, model2_exposures)

In [106]:
model1_exp_loss_no_corr = round(model1_losses_distribution_no_corr.mean(), 4) * 100
model2_exp_loss_no_corr = round(model2_losses_distribution_no_corr.mean(), 4) * 100
result_no_corr.loc[exp_capital_row_name, :] = [model1_exp_loss_no_corr, model2_exp_loss_no_corr]
model1_var_no_corr, model1_ci = project_lib.default_stats.calc_mc_var_and_ci(model1_losses_distribution_no_corr,
                                                                             99.5)
result_no_corr.loc[var_row_name, model1_name] = model1_ci
result_no_corr.loc[req_capital_row_name, model1_name] = model1_var_no_corr - model1_exp_loss_no_corr
model2_var_no_corr, model2_ci = project_lib.default_stats.calc_mc_var_and_ci(model2_losses_distribution_no_corr,
                                                                             99.5)
result_no_corr.loc[var_row_name, model2_name] = model2_ci
result_no_corr.loc[req_capital_row_name, model2_name] = model2_var_no_corr - model2_exp_loss_no_corr

result_no_corr

,Logistic Regression,Our Model
Expected Capital (%),20.71,27.74
VaR (%),20.94 <= 20.97 <= 21.0,27.98 <= 28.02 <= 28.05
Required Capital (%),0.26,0.28


In [107]:
model1_exp_loss_corr = round(model1_losses_distribution_corr.mean(), 4) * 100
model2_exp_loss_corr = round(model2_losses_distribution_corr.mean(), 4) * 100

result_corr.loc[exp_capital_row_name, :] = [model1_exp_loss_corr, model2_exp_loss_corr]
model1_var_corr, model1_ci = project_lib.default_stats.calc_mc_var_and_ci(model1_losses_distribution_corr, 99.5)

result_corr.loc[var_row_name, model1_name] = model1_ci
result_corr.loc[req_capital_row_name, model1_name] = model1_var_corr - model1_exp_loss_corr

model2_var_corr, model2_ci = project_lib.default_stats.calc_mc_var_and_ci(model2_losses_distribution_corr, 99.5)
result_corr.loc[var_row_name, model2_name] = model2_ci
result_corr.loc[req_capital_row_name, model2_name] = model2_var_corr - model2_exp_loss_corr

result_corr

,Logistic Regression,Our Model
Expected Capital (%),20.51,27.5
VaR (%),39.12 <= 40.41 <= 42.1,47.71 <= 49.05 <= 50.78
Required Capital (%),19.9,21.55


In [108]:
real_losses = TEST_DATA.loc[TEST_DATA["default"], "funded_amnt"].sum() / TEST_DATA["funded_amnt"].sum() * 100
project_lib.default_stats.plot_defaults_distributions(model1_name, False, model1_losses_distribution_no_corr,
                                                      model1_exp_loss_no_corr, model1_var_no_corr, real_losses)

In [109]:
project_lib.default_stats.plot_defaults_distributions(model2_name, False, model2_losses_distribution_no_corr,
                                                      model2_exp_loss_no_corr, model2_var_no_corr, real_losses)

In [110]:
project_lib.default_stats.plot_defaults_distributions(model1_name, True, model1_losses_distribution_corr,
                                                      model1_exp_loss_corr, model1_var_corr, real_losses)

In [111]:
project_lib.default_stats.plot_defaults_distributions(model2_name, True, model2_losses_distribution_corr,
                                                      model2_exp_loss_corr, model2_var_corr, real_losses)

# Задание 6

Принимая LGD=100%, рассчитайте для обеих моделей по обучающей выборке порог отсечения, максимизирующий ожидаемую прибыль от кредитования. Постройте графики зависимости ожидаемой прибыли от порога отсечения (для двух моделей в одних осях).

In [100]:
train_pd_calc = TRAIN_FULL_CATBOOST[['funded_amnt', 'installment', 'int_rate', 'term', 'default']].copy(deep=True)

In [101]:
X_test_calc = TRAIN_FULL_CATBOOST.iloc[TRAIN_VAL_WOE.index.values, :]

In [102]:
predicts_test_catboost = TRAIN_TRAIN_CATBOOST_MODEL.predict_proba(
    TRAIN_VAL_CATBOOST[1:].drop('default', axis=1))

In [103]:
predicts_test_logreg = TRAIN_TRAIN_LR_MODEL.predict_proba(TRAIN_VAL_WOE.loc[:, LR_FEATURES])

In [104]:
def calc_pnl(df_for_calc: pd.DataFrame,
                predicts: pd.Series, 
                threshold = 0.5):
    '''
    threshold -- вероятность НЕ-дефолта
    '''
    df = df_for_calc.copy(deep=True)
    df['predict'] = predicts[:, 0]
    df_profit = df[(df.predict > threshold) & (df.default == False)]
    df_loss = df[(df.predict > threshold) & (df.default == True)]
    profit_col = df_profit.installment * df_profit.term.apply(lambda x: int(x[:3])) - df_profit.funded_amnt
    profit = profit_col.sum()
    loss = df_loss.funded_amnt.sum()
    return profit - loss

In [105]:
threshold_space = [threshold for threshold in np.linspace(0.5, 0.99, 500)]

In [106]:
pnl_array_logreg = []
pnl_array_catboost = []
for threshold in threshold_space:
    pnl_array_logreg.append(calc_pnl(X_test_calc, 
                              predicts_test_logreg, 
                              threshold=threshold))
    pnl_array_catboost.append(calc_pnl(
                              X_test_calc,
                              predicts_test_catboost,
                              threshold=threshold))

In [107]:
print(f'Оптимальный порог отсечения для логистической регрессии: {threshold_space[np.argmax(pnl_array_logreg)]: .4f}')
print(f'Ожидаемая прибыль при этом пороге: {np.max(np.array(pnl_array_logreg) / 1000000): .4f} миллионов у.е.\n')

print(f'Оптимальный порог отсечения для гб над решающими деревьями: {threshold_space[np.argmax(pnl_array_catboost)]: .4f}')
print(f'Ожидаемая прибыль при этом пороге: {np.max(np.array(pnl_array_catboost) / 1000000): .4f} миллионов у.е.')

Оптимальный порог отсечения для логистической регрессии:  0.9134
Ожидаемая прибыль при этом пороге:  21.9722 миллионов у.е.

Оптимальный порог отсечения для гб над решающими деревьями:  0.8565
Ожидаемая прибыль при этом пороге:  40.2553 миллионов у.е.


In [108]:
plt.figure(figsize = (20, 15))
plt.grid()
plt.title('Ожидаемая прибыль от кредитования для двух моделей для разных порогов', fontsize=18)
plt.plot(threshold_space, pnl_array_logreg, label='Логистическая регрессия с WOE', marker='o')
plt.plot(threshold_space, pnl_array_catboost, marker='o', label ='ГБ над решающими деревьями')


x_coordinates = [np.min(threshold_space), 1]
y_coordinates_logreg = [np.max(pnl_array_logreg), np.max(pnl_array_logreg)]
y_coordinates_catboost = [np.max(pnl_array_catboost), np.max(pnl_array_catboost)]


plt.plot(x_coordinates, y_coordinates_logreg, 'k')
plt.plot(x_coordinates, y_coordinates_catboost, 'k')

plt.xlabel('Порог отсечения', fontsize=16)
plt.ylabel('Ожидаемая прибыль', fontsize=16)
plt.legend(fontsize = 16)